In [1]:
#Celda para librerías
import sklearn as sk
import sys
print("Python executable:", sys.executable)

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import ExtraTreesClassifier


from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

#app_train_def = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
#app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
#app_train_def_5 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v5.csv')
app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
#app_train_def_7_v2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_def_v7_menor_dimension.csv')

#print(app_train_def_7.info())
#print(app_train_def_7.shape)

Python executable: /home/yeray/miniconda3/envs/tfg-py3.12/bin/python
1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS


In [2]:
X = app_train_def_7.drop(columns = ['TARGET'])
y = app_train_def_7['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [27]:
def objective_completo(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=100),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 100),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 50),
        "max_features": trial.suggest_float("max_features", 0.1, 1.0),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "class_weight": trial.suggest_categorical("class_weight", [None, "balanced"]),
        "random_state": 42,
        "n_jobs": 20
    }
    model = ExtraTreesClassifier(**params)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv)
    
    return scores.mean()


In [ ]:

study_completo = optuna.create_study(direction="maximize")

study_completo.optimize(objective_completo, n_trials=100, show_progress_bar=True)

print("Best trial:", study_completo.best_trial.number)
print("Best value (mean AUC):", study_completo.best_value)
print("Best hyperparams:", study_completo.best_params)

[I 2025-04-07 09:36:29,027] A new study created in memory with name: no-name-844aa747-a3c7-471f-99e9-76d057aceed9


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-04-07 09:50:09,778] Trial 0 finished with value: 0.7480342899234896 and parameters: {'n_estimators': 700, 'max_depth': 8, 'min_samples_split': 46, 'min_samples_leaf': 3, 'max_features': 0.14924364592139455, 'bootstrap': True, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7480342899234896.
[I 2025-04-07 10:24:28,747] Trial 1 finished with value: 0.7546849443534774 and parameters: {'n_estimators': 400, 'max_depth': 10, 'min_samples_split': 22, 'min_samples_leaf': 48, 'max_features': 0.5123051700550051, 'bootstrap': True, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.7546849443534774.
[I 2025-04-07 14:29:05,551] Trial 2 finished with value: 0.7637502517046763 and parameters: {'n_estimators': 1200, 'max_depth': 20, 'min_samples_split': 50, 'min_samples_leaf': 2, 'max_features': 0.38402623180640016, 'bootstrap': False, 'class_weight': 'balanced'}. Best is trial 2 with value: 0.7637502517046763.
[I 2025-04-07 15:02:55,666] Trial 3 finished with value: 0.7345

In [ ]:
import pickle
with open("ExtraTrees_Optuna_Study_v7_completo.pkl", "wb") as f:
    pickle.dump(study_completo, f)
with open("/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/ExtraTrees_Optuna_Study_v7_completo.pkl", "rb") as f:
    loaded_study_completo = pickle.load(f)

In [ ]:
from tabulate import tabulate

trials_df = loaded_study_completo.trials_dataframe()

df_sorted = trials_df.sort_values(by="value", ascending=False)
best_10 = df_sorted.head(10)

print(tabulate(best_10, headers="keys", tablefmt="psql"))

In [ ]:

best_row = best_10.iloc[0]

best_params = best_row.filter(like="params_").to_dict()

best_params_clean = {
    k.replace("params_", ""): v 
    for k, v in best_params.items()
}


final_params = {
    **best_params_clean,
    "random_state": 42,
    "n_jobs": 20
}

extra_model = ExtraTreesClassifier(**final_params)
extra_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = extra_model.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_proba)
print(f"AUC en test: {auc_test:.4f}")
